# ACQUISIZIONE DEI DATI DALLE E-MAIL

In [1]:
class Order():
    def __init__(self, name, size, qty, GC, price):
        self.name = name
        self.size = size
        self.qty = qty
        self.GC = GC
        self.price = price
    
    def __str__(self):
        return f"[{self.name}, {self.qty}, {self.size}, {self.price}, {self.GC}]"

In [2]:
def Extract_quantity(qty_size):
    """Funzione che estrae l'informazione quantità di magliette"""
    sep=" "
    l=qty_size.split(sep)
    for i in range(len(l)):
        boo=l[i].isnumeric()
        if boo==True:
            return l[i]

def Extract_size(qty_size):
     """Funzione che estrae l'informazione taglia"""
    qty_size=qty_size.replace("AzIM", "")
    for i in range(len(qty_size)):
        if i>0 and (qty_size[i]=="S" or qty_size[i]=="M" or qty_size[i]=="L"):
            return qty_size[i]

def Extract_price(price):
     """Funzione che estrae l'informazione prezzo"""
    k="."
    sep=" "
    price=price.replace(",", ".")
    price=price.replace("€", " ")
    l=price.split(sep)
    for i in range(len(l)):
        if k in l[i]:
            return float(l[i])
            
            
def Extract_GC(GC):
     """Funzione che estrae l'informazione Coordinate Geografiche"""
    sep= " "
    GC=GC.replace(GC[len(GC)-1], "")
    l=GC.split(sep)
    l=l[len(l)-1]
    l=l.replace("[","")
    l=l.replace("]","")
    l=l.split(",")
    return(l)

In [21]:
def Read_Orders():
    """Funzione che dalle email acquisisce tutte le 
    informazioni utili e compila una lista di ordini"""
    orders_list=[]
    for i in range(1,421):
        file=open(f'{i}.txt', 'r')
        trash=file.readline() #useless raw
        qty_size=file.readline()
        GC=file.readline()
        price=file.readline()
        trash=file.readline() #useless raw
        name=file.readline()
        current_order=Order(name, Extract_size(qty_size), Extract_quantity(qty_size), Extract_GC(GC), Extract_price(price))
        orders_list.append(current_order)
    return orders_list      

# ELABORAZIONE DEI DATI

In [42]:
def Distance(A,B):
    """Dati due punti A,B in coordinate GCS restituisce la distanza A-B in km"""
    d=((float(B[0])-A[0])**2 + (float(B[1])-A[1])**2)**(1/2)
    return d

#Funzione Distance che tiene conto della curvatura terrestre
#def Distance(A,B):
#    import math
#    R=6372.795477598
#    latA=math.radians(float(A[0]))
#    latB=math.radians(float(B[0]))
#    lonA=math.radians(float(A[1]))
#    lonB=math.radians(float(B[1]))
#    d=R*(math.acos(math.sin(latA)*math.sin(latB)+math.cos(latA)*math.cos(latB)*math.cos(lonA-lonB)))
#    return float(d)
    

def Stimate_Earnings(orders_list, selling_price, production_costs):
    """Stima il ricavo di tutte le vendite accettabili a 
    un determinato prezzo fissato"""
    tot=0.0
    for i in range(len(orders_list)):
        if selling_price <= orders_list[i].price:
            tot=tot+((selling_price - production_costs)*float(orders_list[i].qty))       
    return tot

def Stimate_Earnings_with_shipping(orders_list, selling_price, HeadQuarter_GC, shipping_costs, production_costs):
    """Stima il ricavo di tutte le vendite accettabili a 
    un determinato prezzo fissato tenendo conto dei costi di spedizione"""
    tot=0.0
    for i in range(len(orders_list)):
        if selling_price <= orders_list[i].price:
            d=Distance(HeadQuarter_GC, orders_list[i].GC)
            if int(orders_list[i].qty)<100:
                tot=tot+((selling_price - production_costs)*float(orders_list[i].qty))-(d*shipping_costs[0])
            else:
                tot=tot+((selling_price - production_costs)*float(orders_list[i].qty))-(d*shipping_costs[1])
    return tot

In [43]:
def Best_price_calculation(orders_list, HeadQuarter_GC, shipping_costs, production_costs):
    """Funzione che stima i prezzi di vendita ottimale 
    per le magliette (sia crudo sia tenendo conto dei costi di spedizione)"""
    lp=[]
    potential_earnings=[]
    potential_earnings_with_shipping=[]
    best_price=0
    best_price_with_shipping=0
    cont=0.0
    ME=0
    MEWS=0
    bp=[]
    
    for i in range(len(orders_list)):
        lp.append(orders_list[i].price)
        
    M=max(lp)
    
    while cont<=M:
        t=Stimate_Earnings(orders_list, cont, production_costs)
        k=Stimate_Earnings_with_shipping(orders_list, cont, HeadQuarter_GC, shipping_costs, production_costs)
        potential_earnings.append(t)
        potential_earnings_with_shipping.append(k)
        cont=cont+0.01
        
    for i in range (len(potential_earnings)):
        if potential_earnings[i]>=ME:
            ME=potential_earnings[i]
            
    for i in range (len(potential_earnings)):
        if ME==potential_earnings[i]:
            best_price=i*0.01
            bp.append(best_price)
        
    for i in range (len(potential_earnings_with_shipping)):
        if potential_earnings_with_shipping[i]>=MEWS:
            MEWS=potential_earnings_with_shipping[i]
            
    for i in range (len(potential_earnings_with_shipping)):
        if MEWS==potential_earnings_with_shipping[i]:
            best_price_with_shipping=i*0.01
            bp.append(best_price_with_shipping)
    return bp

def To_Do_List(orders_list, bpws, HeadQuarter_GC, shipping_costs, production_costs):
    """Funzione che stila la lista degli ordini accettati da eseguire, da quello 
    che porterebbe a un maggior profitto fino a quello col minor profitto"""
    Executable_order_list=[]
    tot=0.0
    Ordered_order_list=[]
    for i in range(len(orders_list)):
        if orders_list[i].price>=bpws:
            Executable_order_list.append(orders_list[i])
    for _ in range(len(Executable_order_list)):
        M=0.0
        for i in range(len(Executable_order_list)):
            d=Distance(HeadQuarter_GC, Executable_order_list[i].GC)
            if int(Executable_order_list[i].qty)<100:
                tot=(bpws-production_costs)*float(Executable_order_list[i].qty)-(d*shipping_costs[0])
            else:
                tot=(bpws-production_costs)*float(Executable_order_list[i].qty)-(d*shipping_costs[1])
            if tot>=M:
                M=tot
                maxindex=i
        Ordered_order_list.append(Executable_order_list[maxindex])
        Executable_order_list.remove(Executable_order_list[maxindex])
    return Ordered_order_list

# MAIN

In [44]:
#VARIABILI DA MODIFICARE
HeadQuarter_GC=[45.47946, 9.22837]
shipping_costs=[0.10, 0.05]
production_costs=6.00

#FUNZIONI
orders_list = Read_Orders()
best_selling_price = Best_price_calculation(orders_list, HeadQuarter_GC, shipping_costs, production_costs)
final_list = To_Do_List(orders_list, best_selling_price[1], HeadQuarter_GC, shipping_costs, production_costs)

#STAMPA
print(f"Il prezzo di vendita ottimale senza tener conto dei costi di spedizione è {best_selling_price[0]}€")
print(f"\nIl prezzo di vendita ottimale tenendo conto dei costi di spedizione è {best_selling_price[1]}€")
print("\n\nLista delle richieste accettate in ordine decrescente di profitto:\n")
for i in range(len(final_list)):
    print(f"{i+1})")
    print(final_list[i])
    print("______________________________________________________________________________________\n")

Il prezzo di vendita ottimale senza tener conto dei costi di spedizione è 13.22€

Il prezzo di vendita ottimale tenendo conto dei costi di spedizione è 13.22€


Lista delle richieste accettate in ordine decrescente di profitto:

1)
[JP Iacovelli Azienda Agricola, 497, S, 17.92, ['45.058065', '6.422715']]
______________________________________________________________________________________

2)
[Francesco Sosta IB, 494, L, 15.6, ['44.958786', '10.107809']]
______________________________________________________________________________________

3)
[Manuel Carrara Franchising, 492, S, 19.85, ['44.811402', '11.070849']]
______________________________________________________________________________________

4)
[Maurizio Cerisola Frumento, 492, S, 16.34, ['45.149412', '12.169453']]
______________________________________________________________________________________

5)
[Gianmarco Bressan Company, 491, S, 19.79, ['45.661732', '10.428504']]
____________________________________________________